In [1]:
%reload_ext autoreload
%autoreload 2

In [17]:
import numpy as np
import os
import csv
from scipy import stats
import pickle
import matplotlib
import matplotlib.pyplot as plt
from gensim.corpora import Dictionary
from biohasher import Biohasher

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [18]:
synapse_file = '../data/synapses/OWT_hid_400_W_11_LR_0.0002_14.npy'
tokenizer_file = '../data/tokenizer/gensim1_patched.dict'
stopword_file = '../data/tokenizer/exceptional_ids_terrier_stop.npy'
normalize_synapses = True
biohasher = Biohasher(synapse_file, tokenizer_file, stopword_file=stopword_file, normalize_synapses=normalize_synapses)

In [19]:
biohasher.synapses

Loading synapses...


array([[-4.43565028e-07,  1.36138871e-02,  6.29986811e-04, ...,
         4.43897079e-07,  1.14646364e-05, -4.43969441e-07],
       [ 4.49971623e-07,  2.36383919e-02,  6.20333652e-04, ...,
        -4.50004052e-07,  4.50628761e-07,  2.90354237e-05],
       [-4.35325774e-07,  1.04162535e-02,  1.46046351e-03, ...,
         4.35665356e-07,  1.21701187e-05,  4.35689259e-07],
       ...,
       [ 4.47546086e-07,  4.02045669e-03,  1.21266430e-03, ...,
        -4.49028761e-07,  4.47576895e-07, -4.49313887e-07],
       [ 5.98082863e-07,  6.22183830e-03,  9.74089140e-04, ...,
         7.94543394e-06,  1.78764858e-05,  6.68392829e-07],
       [ 9.68047516e-07,  3.38649952e-05,  2.12665577e-03, ...,
        -9.82031565e-07,  2.41769976e-05,  2.87291659e-05]], dtype=float32)

In [20]:
N_voc=20000
N=2*N_voc
hid=400
prec=1.0e-32
p=2

synapses=np.load('../data/synapses/OWT_hid_400_W_11_LR_0.0002_14.npy')
print(synapses.shape, N)

def normalize(syn):
    [K,N]=syn.shape
    nc=np.power(np.sum(syn**p,axis=1),1/p).reshape(K,1)
    syn=syn/np.tile(nc+prec,(1,N))
    return syn

synapses=normalize(synapses)

(400, 40000) 40000


In [21]:
#VOC = Dictionary.load('/REMOTE/OpenWebText/models/gensim1_patched.dict')
VOC = Dictionary.load('../data/tokenizer/gensim1_patched.dict')
exceptional_tokens=np.load('../data/tokenizer/exceptional_ids_terrier_stop.npy')    

N_VOC=len(VOC)
print(N_VOC)

tok2id={}
for i in range(N_VOC):
    tok2id[VOC[i]]=i

20000


In [22]:
#phrase = 'boat on the bank of the river'
#phrase = 'apple latest iphone'
#phrase = 'money in bank checking account'
#phrase = 'the company stock is training high'
#phrase = 'sweet crispy apple pie'
phrase = 'But that dialogue did not reflect the state of her marriage Kidman told the magazine'
#phrase = 'nation senior military leaders should quarantine after they were advised that Admiral Charles Ray with whom they had met with at a Pentagon meeting had tested positive for the virus'
#phrase = 'my research focuses on the computational properties of neural networks'
#phrase = 'trump administration campaign rally in oklahoma'
#phrase = 'ibm corporation to acquire opensource software startup'
#phrase = 'local government officials responded promptly to protests'
#phrase = 'influenza virus outbreak in public schools'

# Tokenize the phrase
v = np.zeros((N,1))
for w in phrase.split(' '):
    w = w.lower()
    if w in tok2id.keys():
        print(w, tok2id[w], tok2id[w] not in exceptional_tokens)
        if tok2id[w] not in exceptional_tokens:
            v[tok2id[w],0] = 1.
        
        
        
print(np.nonzero(v))        
nc = np.sqrt(np.sum(v*v))
v = v/nc

but 170 False
that 31 False
dialogue 3193 True
did 259 False
not 24 False
reflect 2861 True
the 32 False
state 213 True
of 25 False
her 460 False
marriage 5302 True
told 845 True
the 32 False
magazine 1556 True
(array([ 213,  845, 1556, 2861, 3193, 5302]), array([0, 0, 0, 0, 0, 0]))


In [24]:
def softmax(x: np.array, beta=1.0):
    v = np.exp(beta*x)
    return v / np.sum(v)

In [25]:
def show_head_contribution(ind_head, beta=10.0, beta_tar=800.0, beta_con=10.0):
    OV = np.dot(synapses,v)
    SM = softmax(OV, beta)
    mem_ordered = np.argsort(-SM[:,0])
    print('Contribution of the chosen head', 100*SM[mem_ordered[ind_head],0])


    RHS_tar= synapses[mem_ordered[ind_head],N_voc:]
    RHS_con= synapses[mem_ordered[ind_head],:N_voc]

    Z_out = np.sum(np.exp(beta_tar*RHS_tar))
    RHS_tar_norm = np.exp(beta_tar*RHS_tar)/Z_out

    N_show = 20
    print('TARGET')
    for ID in np.argsort(-RHS_tar_norm)[:N_show]:
        print(ID, VOC[ID], int(ID) not in exceptional_tokens, 100*RHS_tar_norm[ID])

    Z_out = np.sum(np.exp(beta_con*RHS_con))
    RHS_con_norm = np.exp(beta_con*RHS_con)/Z_out

    N_show = 20
    print('CONTEXT')
    for ID in np.argsort(-RHS_con_norm)[:N_show]:
        print(ID, VOC[ID], int(ID) not in exceptional_tokens, 100*RHS_con_norm[ID])

interact(show_head_contribution, 
         ind_head=widgets.IntSlider(min=0, max=400, step=1, value=0), 
         beta=widgets.FloatSlider(min=0.05, max=20.0, step=0.2, value=10.0),
         beta_tar=widgets.FloatSlider(min=10, max=1000.0, step=20, value=800.0),
         beta_con=widgets.FloatSlider(min=0.5, max=50, step=2, value=10),
        )

interactive(children=(IntSlider(value=0, description='ind_head', max=400), FloatSlider(value=10.0, description…

<function __main__.show_head_contribution(ind_head, beta=10.0, beta_tar=800.0, beta_con=10.0)>

In [27]:
beta = 10.0
OV = np.dot(synapses,v)
Z = np.sum(np.exp(beta*OV))
SM = np.exp(beta*OV)/Z
mem_ordered = np.argsort(-SM[:,0])

ind_head = 1
print('Contribution of the chosen head', 100*SM[mem_ordered[ind_head],0])

RHS_tar= synapses[mem_ordered[ind_head],N_voc:]
# RHS_tar= synapses[ind_head,N_voc:]
RHS_con= synapses[mem_ordered[ind_head],:N_voc]

beta_out_tar = 800.0
Z_out = np.sum(np.exp(beta_out_tar*RHS_tar))
RHS_tar_norm = np.exp(beta_out_tar*RHS_tar)/Z_out

N_show = 20
print('TARGET')
for ID in np.argsort(-RHS_tar_norm)[:N_show]:
    print(ID, VOC[ID], int(ID) not in exceptional_tokens, 100*RHS_tar_norm[ID])

beta_out_con = 10.0
Z_out = np.sum(np.exp(beta_out_con*RHS_con))
RHS_con_norm = np.exp(beta_out_con*RHS_con)/Z_out
    
N_show = 20
print('CONTEXT')
for ID in np.argsort(-RHS_con_norm)[:N_show]:
    print(ID, VOC[ID], int(ID) not in exceptional_tokens, 100*RHS_con_norm[ID])
    

Contribution of the chosen head 7.440782072503222
TARGET
859 islamic True 35.08584499359131
2844 secretary True 8.452624827623367
15737 kerry True 1.1010242626070976
1538 department True 0.8490746840834618
12319 tillerson True 0.4403265193104744
213 state True 0.27652173303067684
9478 rex True 0.1497171469964087
3072 university True 0.0496988242957741
12261 penn True 0.0492349558044225
13011 ohio True 0.04402336198836565
1246 justice True 0.04310351505409926
858 iraq True 0.03920515882782638
15083 levant True 0.0367013766663149
3638 clinton True 0.03553197311703116
3640 hillary True 0.03478354774415493
4292 militants True 0.03137144376523793
15613 legislatures True 0.03115883155260235
5425 legislature True 0.028788219788111746
4127 affairs True 0.027922206209041178
894 local True 0.02577339473646134
CONTEXT
213 state True 23.084934055805206
859 islamic True 0.0672393711283803
1538 department True 0.03881174779962748
2844 secretary True 0.024974113330245018
12319 tillerson True 0.005975

In [15]:
biohasher.get_mem_concepts(1)

[{'token': 'years', 'contribution': 0.011394077},
 {'token': 'weeks', 'contribution': 0.0013536372},
 {'token': 'months', 'contribution': 0.00065373187},
 {'token': '<NUM>', 'contribution': 0.0004905109},
 {'token': 'spate', 'contribution': 0.00040324018},
 {'token': 'decades', 'contribution': 0.00021578977},
 {'token': 'developments', 'contribution': 0.00019786299},
 {'token': 'poll', 'contribution': 0.00019749136},
 {'token': 'days', 'contribution': 0.00019449818},
 {'token': 'study', 'contribution': 0.00018858637},
 {'token': 'report', 'contribution': 0.00018803438},
 {'token': 'interview', 'contribution': 0.00015840567},
 {'token': 'history', 'contribution': 0.00015452725},
 {'token': 'graduates', 'contribution': 0.0001543284},
 {'token': 'survey', 'contribution': 0.00015281916},
 {'token': 'polls', 'contribution': 0.00015257808},
 {'token': 'attacks', 'contribution': 0.00014666107},
 {'token': 'according', 'contribution': 0.00013803918},
 {'token': 'events', 'contribution': 0.0001

In [9]:
@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

interactive(children=(Checkbox(value=True, description='x'), FloatSlider(value=1.0, description='y', max=3.0, …